# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
%pip install hvplot

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install gmaps

In [23]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import geoapify_key

In [2]:
from pathlib import Path
import os

In [3]:
import gmaps

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir) 
# data_dir = os.path.join(parent_dir, "output_data")

# file_path = os.path.join(data_dir, 'cities.csv')

# city_data_df.to_csv(file_path, index=False)

city_to_load = os.path.join(parent_dir, "output_data", "cities.csv")
city_data_df = pd.read_csv(city_to_load)

# print(f"CSV file saved to: {file_path}")
# city_data_df = pd.read_csv("output_data/cities.csv")

# # Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,praia da vitoria,38.7333,-27.0667,22.14,73,40,4.14,PT,2024-06-08 09:18:43
1,grytviken,-54.2811,-36.5092,-2.51,83,18,2.72,GS,2024-06-08 09:18:43
2,adamstown,-25.0660,-130.1015,22.58,82,63,4.66,PN,2024-06-08 09:18:43
3,san felipe,10.3399,-68.7425,34.57,38,21,3.11,VE,2024-06-08 09:18:43
4,lander,42.8330,-108.7307,20.62,52,0,2.57,US,2024-06-08 09:18:43


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 50) & (city_data_df["Wind Speed"] < 30) & (city_data_df["Cloudiness"]<40)& (city_data_df["Humidity"]<30)]


# Drop any rows with null values
filtered_df = filtered_df.dropna()


# Display sample data
# YOUR CODE HERE
filtered_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,gao,16.6362,1.6370,42.84,14,4,2.62,ML,2024-06-08 09:18:51
69,chardara,41.2547,67.9692,29.51,28,0,2.89,KZ,2024-06-08 09:18:55
72,murzuq,25.9155,13.9184,36.86,8,0,4.83,LY,2024-06-08 09:18:55
81,coxim,-18.5067,-54.7600,34.06,24,7,4.08,BR,2024-06-08 09:18:57
110,luau,-10.7073,22.2247,25.33,26,9,1.36,AO,2024-06-08 09:19:01


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = filtered_df.iloc[:,[0,1,2,4,7]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "
hotel_df.head()


,City,Lat,Lng,Humidity,Country,Hotel Name
47,gao,16.6362,1.6370,14,ML,
69,chardara,41.2547,67.9692,28,KZ,
72,murzuq,25.9155,13.9184,8,LY,
81,coxim,-18.5067,-54.7600,24,BR,
110,luau,-10.7073,22.2247,26,AO,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    'filter':'',
    'bias':'',
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample dat
#

Starting hotel search
gao - nearest hotel: No hotel found
chardara - nearest hotel: Бақдаулет
murzuq - nearest hotel: فندق باريس - مرزق
coxim - nearest hotel: Avendita
luau - nearest hotel: No hotel found
shakawe - nearest hotel: EdenWood guesthouse
zamania - nearest hotel: No hotel found
urumqi - nearest hotel: 百时快捷酒店红山店
zouerate - nearest hotel: فندق تيرس زمور
chapada dos guimaraes - nearest hotel: Pousada Rios
creel - nearest hotel: Hotel Cascada inn
karaj - nearest hotel: هتل مرمر
yoloten - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
tir pul - nearest hotel: No hotel found
nahiyat ash shinafiyah - nearest hotel: No hotel found
ak'ordat - nearest hotel: Omal Hotel
jaisalmer - nearest hotel: Jeet Mahal
kayenta - nearest hotel: Wetherill Inn
al kharijah - nearest hotel: Kharga Hotel
arraias - nearest hotel: Hotel Japão
istgah-e rah ahan-e garmsar - nearest hotel: هتل شهر آفتاب
smara - nearest hotel: Hôtel Amine فندق ا

In [15]:
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
47,gao,16.6362,1.6370,14,ML,No hotel found
69,chardara,41.2547,67.9692,28,KZ,Бақдаулет
72,murzuq,25.9155,13.9184,8,LY,فندق باريس - مرزق
81,coxim,-18.5067,-54.7600,24,BR,Avendita
110,luau,-10.7073,22.2247,26,AO,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%pip install cartopy
%pip install geoviews
%pip install  pyproj

  Using cached pyshp-2.3.1-py2.py3-none-any.whl.metadata (55 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 9.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.3 MB/s eta 0:00:0000:0100:01
Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Using cached geoviews-1.12.0-py3-none-any.whl.metadata (12 kB)
Using cached geoviews-1.12.0-py3-none-any.whl (557 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import cartopy
import geoviews
import pyproj

In [20]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)


# Display the map
# YOUR CODE HERE
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)